In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_test_prepro = pd.read_csv('data/preprocessed_test_data_with_date_test.csv')
df_aircraft_model = pd.read_csv('data/full_aircraft_model_mapping_CORRECTED.csv')
df_passenger = pd.read_csv('data/passenger_data.csv')
airports_coutry_city = pd.read_csv("data/airport_country_city_mapping.csv")

In [3]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,arr_time_of_day,route,departure_date,arrival_date,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,morning,DJE → TUN,2016-05-04,2016-05-04,21.2,0.0,12.7,18.7,0.0,21.10
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,evening,TUN → BKO,2016-05-05,2016-05-05,18.4,0.0,9.7,34.4,0.0,13.10
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,afternoon,FRA → TUN,2016-05-06,2016-05-06,17.0,0.0,7.9,19.7,0.0,18.20
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,afternoon,BEY → TUN,2016-05-11,2016-05-11,21.3,0.0,10.3,23.1,0.0,15.10
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,afternoon,ORY → MIR,2016-05-11,2016-05-11,14.6,4.0,4.8,19.9,0.0,13.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,afternoon,TUN → NCE,2018-09-12,2018-09-12,25.9,0.0,8.6,24.4,0.0,10.50
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,night,TUN → TUN,2018-09-27,2018-09-28,21.3,14.5,18.0,21.3,0.0,16.10
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,morning,SJJ → TUN,2018-09-03,2018-09-03,18.9,0.0,5.1,27.1,0.0,9.20
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,afternoon,TUN → DJE,2018-09-15,2018-09-15,26.4,0.8,8.4,27.9,0.0,14.80


In [4]:
df_test_prepro.isnull().sum()  # Check for any remaining missing values

id                         0
departure_point            0
arrival_point              0
departure_time             0
arrival_time               0
flight_status              0
aircraft_code              0
duration                   0
departure_time_formated    0
arrival_time_formated      0
dep_hour                   0
dep_day                    0
dep_month                  0
dep_dayofweek              0
dep_quarter                0
dep_season                 0
dep_is_weekend             0
dep_time_of_day            0
arr_hour                   0
arr_day                    0
arr_month                  0
arr_dayofweek              0
arr_quarter                0
arr_season                 0
arr_is_weekend             0
arr_time_of_day            0
route                      0
departure_date             0
arrival_date               0
dep_temp                   0
dep_precip                 0
dep_wind                   0
arr_temp                   0
arr_precip                 0
arr_wind      

In [5]:
df_test_prepro['departure_time'] = pd.to_datetime(df_test_prepro['departure_time'])
df_test_prepro['departure_date'] = df_test_prepro['departure_time'].dt.date

# Map aircraft model to main df

In [6]:
# Map aircraft_model to df_test_prepro using aircraft_code
df_test_prepro['aircraft_model'] = df_test_prepro['aircraft_code'].map(
    df_aircraft_model.set_index('aircraft_code')['aircraft_model']
)

In [6]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,route,departure_date,arrival_date,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,aircraft_model
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,DJE → TUN,2016-05-04,2016-05-04,21.2,0.0,12.7,18.7,0.0,21.10,Airbus A320
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,TUN → BKO,2016-05-05,2016-05-05,18.4,0.0,9.7,34.4,0.0,13.10,Airbus A320-200
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,FRA → TUN,2016-05-06,2016-05-06,17.0,0.0,7.9,19.7,0.0,18.20,Airbus A320
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,BEY → TUN,2016-05-11,2016-05-11,21.3,0.0,10.3,23.1,0.0,15.10,Airbus A318
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,ORY → MIR,2016-05-11,2016-05-11,14.6,4.0,4.8,19.9,0.0,13.65,Boeing 737-600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,TUN → NCE,2018-09-12,2018-09-12,25.9,0.0,8.6,24.4,0.0,10.50,Airbus A320-200
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,TUN → TUN,2018-09-27,2018-09-28,21.3,14.5,18.0,21.3,0.0,16.10,Airbus A320
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,SJJ → TUN,2018-09-03,2018-09-03,18.9,0.0,5.1,27.1,0.0,9.20,Bombardier CRJ-900
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,TUN → DJE,2018-09-15,2018-09-15,26.4,0.8,8.4,27.9,0.0,14.80,ATR 72


# Map Passenger data to main df

In [7]:
# Ensure departure_date is datetime
df_test_prepro['departure_date'] = pd.to_datetime(df_test_prepro['departure_date'])

# Extract year from departure_date
df_test_prepro['year'] = df_test_prepro['departure_date'].dt.year

# Melt df_passenger to long format for easy merging
df_passenger_long = df_passenger.melt(
    id_vars=['IATA_Code'],
    value_vars=['2016 Passengers', '2017 Passengers', '2018 Passengers'],
    var_name='year_col',
    value_name='num_passenger_year'
)
df_passenger_long['year'] = df_passenger_long['year_col'].str.extract(r'(\d{4})').astype(int)

# Merge passenger info into flight data (use 'departure_point' instead of 'IATA_Code')
df_test_prepro = df_test_prepro.merge(
    df_passenger_long[['IATA_Code', 'year', 'num_passenger_year']],
    left_on=['departure_point', 'year'],
    right_on=['IATA_Code', 'year'],
    how='left'
)

# Drop the temporary columns if you don't need them
df_test_prepro = df_test_prepro.drop(columns=['year', 'IATA_Code'])

In [8]:
# Clean and convert 'num_passenger_year' to integer
df_test_prepro['num_passenger_year'] = (
    df_test_prepro['num_passenger_year']
    .replace(',', '', regex=True)
    .astype(float)
    .astype('Int64')
)

In [9]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,departure_date,arrival_date,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,aircraft_model,num_passenger_year
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,2016-05-04,2016-05-04,21.2,0.0,12.7,18.7,0.0,21.10,Airbus A320,993429
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,2016-05-05,2016-05-05,18.4,0.0,9.7,34.4,0.0,13.10,Airbus A320-200,5200000
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,2016-05-06,2016-05-06,17.0,0.0,7.9,19.7,0.0,18.20,Airbus A320,60792308
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,2016-05-11,2016-05-11,21.3,0.0,10.3,23.1,0.0,15.10,Airbus A318,7370128
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,2016-05-11,2016-05-11,14.6,4.0,4.8,19.9,0.0,13.65,Boeing 737-600,31340357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,2018-09-12,2018-09-12,25.9,0.0,8.6,24.4,0.0,10.50,Airbus A320-200,6000000
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,2018-09-27,2018-09-28,21.3,14.5,18.0,21.3,0.0,16.10,Airbus A320,6000000
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,2018-09-03,2018-09-03,18.9,0.0,5.1,27.1,0.0,9.20,Bombardier CRJ-900,1046635
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,2018-09-15,2018-09-15,26.4,0.8,8.4,27.9,0.0,14.80,ATR 72,6000000


# Map Distance km data to main df

In [10]:
import airportsdata

# Load the airports database (IATA codes)
airports = airportsdata.load('IATA')

def get_lat_lon(iata):
    info = airports.get(iata)
    if info:
        return pd.Series({'lat': info['lat'], 'lon': info['lon']})
    else:
        return pd.Series({'lat': None, 'lon': None})

# Populate departure coordinates
dep_coords = df_test_prepro['departure_point'].apply(get_lat_lon)
df_test_prepro['dep_lat'] = dep_coords['lat']
df_test_prepro['dep_long'] = dep_coords['lon']

# Populate arrival coordinates
arr_coords = df_test_prepro['arrival_point'].apply(get_lat_lon)
df_test_prepro['arr_lat'] = arr_coords['lat']
df_test_prepro['arr_long'] = arr_coords['lon']

# Fill SXF manually if still missing
sxf_lat, sxf_long = 52.380001, 13.522500
df_test_prepro.loc[df_test_prepro['departure_point'] == 'SXF', ['dep_lat', 'dep_long']] = [sxf_lat, sxf_long]
df_test_prepro.loc[df_test_prepro['arrival_point'] == 'SXF', ['arr_lat', 'arr_long']] = [sxf_lat, sxf_long]

In [11]:
# Haversine formula to calculate distance in km
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Calculate the distance for each row
df_test_prepro['distance_km'] = haversine(
    df_test_prepro['dep_lat'],
    df_test_prepro['dep_long'],
    df_test_prepro['arr_lat'],
    df_test_prepro['arr_long']
)

df_test_prepro['distance_km'] = (df_test_prepro['distance_km'].round(2) * 100).fillna(0).astype(int)

In [12]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,arr_temp,arr_precip,arr_wind,aircraft_model,num_passenger_year,dep_lat,dep_long,arr_lat,arr_long,distance_km
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,18.7,0.0,21.10,Airbus A320,993429,33.8750,10.77550,36.8510,10.22720,33463
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,34.4,0.0,13.10,Airbus A320-200,5200000,36.8510,10.22720,12.5335,-7.94994,325505
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,19.7,0.0,18.20,Airbus A320,60792308,50.0264,8.54313,36.8510,10.22720,147123
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,23.1,0.0,15.10,Airbus A318,7370128,33.8209,35.48840,36.8510,10.22720,230919
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,19.9,0.0,13.65,Boeing 737-600,31340357,48.7253,2.35944,35.7581,10.75470,159661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,24.4,0.0,10.50,Airbus A320-200,6000000,36.8510,10.22720,43.6584,7.21587,79877
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,21.3,0.0,16.10,Airbus A320,6000000,36.8510,10.22720,36.8510,10.22720,0
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,27.1,0.0,9.20,Bombardier CRJ-900,1046635,43.8246,18.33150,36.8510,10.22720,103486
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,27.9,0.0,14.80,ATR 72,6000000,36.8510,10.22720,33.8750,10.77550,33463


# Map expected duration data to main df

In [13]:
# TEMP NOT ACTIVE UNTIL WEATHER COLUMNS FIXED
route_durations = df_test_prepro.groupby('route')['duration'].median().reset_index()
route_durations.rename(columns={'duration': 'expected_duration'}, inplace=True)
route_durations.sort_values(by='expected_duration', ascending=False, inplace=True)

In [14]:
# TEMP INACTIVE -Merge back into the original dataframe
df_train_prepro = df_test_prepro.merge(route_durations, on='route', how='left')

In [16]:
# Merge expected_duration into df_test_prepro if not already present
if 'expected_duration' not in df_test_prepro.columns:
	df_test_prepro = df_test_prepro.merge(route_durations, on='route', how='left')

#CREATE DELAY RELATIVE TO EXPECTED AND DURATION RATIO COLUMNS
df_test_prepro['delay_relative_to_expected'] = df_test_prepro['duration'] - df_test_prepro['expected_duration']
df_test_prepro['duration_ratio'] = df_test_prepro['duration'] / df_test_prepro['expected_duration']

In [17]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,aircraft_model,num_passenger_year,dep_lat,dep_long,arr_lat,arr_long,distance_km,expected_duration,delay_relative_to_expected,duration_ratio
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,Airbus A320,993429,33.8750,10.77550,36.8510,10.22720,33463,3600.0,-600.0,0.833333
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,Airbus A320-200,5200000,36.8510,10.22720,12.5335,-7.94994,325505,17100.0,0.0,1.000000
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,Airbus A320,60792308,50.0264,8.54313,36.8510,10.22720,147123,8700.0,0.0,1.000000
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,Airbus A318,7370128,33.8209,35.48840,36.8510,10.22720,230919,12600.0,0.0,1.000000
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,Boeing 737-600,31340357,48.7253,2.35944,35.7581,10.75470,159661,9300.0,600.0,1.064516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,Airbus A320-200,6000000,36.8510,10.22720,43.6584,7.21587,79877,5400.0,0.0,1.000000
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,Airbus A320,6000000,36.8510,10.22720,36.8510,10.22720,0,10800.0,0.0,1.000000
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,Bombardier CRJ-900,1046635,43.8246,18.33150,36.8510,10.22720,103486,6600.0,0.0,1.000000
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,ATR 72,6000000,36.8510,10.22720,33.8750,10.77550,33463,3600.0,0.0,1.000000


In [18]:
# Drop any 'Unnamed' columns from df_train_prepro
df_test_prepro = df_test_prepro.loc[:, ~df_test_prepro.columns.str.contains('^Unnamed')]

In [19]:
# Add a 'target' column with empty values (NaN) for the test set
df_test_prepro['target'] = np.nan

# Map holiday columns to main df

In [20]:
departure_date_country = df_test_prepro[["departure_time", "departure_point"]].sort_values(by=["departure_time", "departure_point"])

In [14]:
airports_coutry_city

,IATA_Code,Country,City,2016 Passengers,2017 Passengers,2018 Passengers
0,AAE,Algeria,Annaba,"287,421","302,492","318,116"
1,AAL,Denmark,Aalborg,"1,518,879","1,607,987","1,643,303"
2,ABJ,Côte d’Ivoire,Abidjan,"1,860,000","2,080,000","2,200,000"
3,ADB,Turkey,Izmir,"11,951,183","12,400,000","13,100,000"
4,AHU,Morocco,Al Hoceima,"92,083","122,036","179,355"
...,...,...,...,...,...,...
125,VKO,Russia,Moscow,"16,700,000","18,100,000","21,400,000"
126,VNO,Lithuania,Vilnius,"224,157","235,365","247,133"
127,VRN,Italy,Verona,"2,750,000","3,100,000","3,360,000"
128,YUL,Canada,Montreal,"10,000,000","10,500,000","11,000,000"


In [21]:
df_test_prepro = df_test_prepro.merge(
    airports_coutry_city[['IATA_Code', 'Country', 'City']],
    left_on='departure_point',
    right_on='IATA_Code',
    how='left'
).drop(columns=['IATA_Code'])

In [23]:
from datetime import datetime
import holidays

# Ensure departure_time is parsed to datetime
df_test_prepro['departure_time'] = pd.to_datetime(df_test_prepro['departure_time'])
df_test_prepro['departure_date'] = df_test_prepro['departure_time'].dt.date

# Mapping for countries to ISO codes supported by the holidays package
country_code_map = {
    'United Kingdom': 'GB',
    'Côte d’Ivoire': 'CI',
    'DR Congo': 'CD',
    'Russia': 'RU',
    'Turkey': 'TR',
    'France': 'FR',
    'Germany': 'DE',
    'Italy': 'IT',
    'Spain': 'ES',
    'Canada': 'CA',
    'Portugal': 'PT',
    'Austria': 'AT',
    'Belgium': 'BE',
    'Switzerland': 'CH',
    'Greece': 'GR',
    'Netherlands': 'NL',
    'Luxembourg': 'LU',
    'Ireland': 'IE',
    'Denmark': 'DK',
    'Norway': 'NO',
    'Sweden': 'SE',
    'Finland': 'FI',
    'Czech Republic': 'CZ',
    'Poland': 'PL',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Hungary': 'HU',
    'Romania': 'RO',
    'Lithuania': 'LT',
    'Latvia': 'LV',
    'Estonia': 'EE',
    'Ukraine': 'UA',
    'Serbia': 'RS',
    'Bosnia and Herzegovina': 'BA'
}

# Build holiday sets for countries
country_holidays = {}

for country in df_test_prepro['Country'].dropna().unique():
    code = country_code_map.get(country)
    if code:
        country_holidays[country] = holidays.CountryHoliday(code, years=[2016, 2017, 2018])
    else:
        country_holidays[country] = set()

# Function to check if a date is a holiday in a given country
def is_holiday(row):
    country = row['Country']
    date = row['departure_date']
    return 1 if date in country_holidays.get(country, set()) else 0

# Apply function
df_test_prepro['is_holiday'] = df_test_prepro.apply(is_holiday, axis=1)

In [24]:
from datetime import datetime

# Function to get the length of the consecutive holiday period for a given date and country
def holiday_length(row):
    if row['is_holiday'] != 1:
        return 0
    country = row['Country']
    date = row['departure_date']
    holidays_set = set(country_holidays.get(country, {}).keys())
    if not holidays_set:
        return 1  # If no holidays found, treat as single-day holiday

    # Convert date to datetime.date if it's a string
    if isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d").date()

    # Count consecutive days before and after
    length = 1
    prev_date = date
    next_date = date

    # Check previous days
    while True:
        prev_date = prev_date - pd.Timedelta(days=1)
        if prev_date in holidays_set:
            length += 1
        else:
            break

    # Check next days
    while True:
        next_date = next_date + pd.Timedelta(days=1)
        if next_date in holidays_set:
            length += 1
        else:
            break

    return length

# Apply the function to the DataFrame
df_test_prepro['holiday_length'] = df_test_prepro.apply(holiday_length, axis=1)

# Show a sample of holidays with their lengths
df_test_prepro[df_test_prepro['is_holiday'] == 1][['departure_date', 'Country', 'holiday_length']]

,departure_date,Country,holiday_length
59,2016-05-05,France,1
60,2016-05-05,France,1
78,2016-05-05,France,1
88,2016-05-05,France,1
90,2016-05-05,France,1
...,...,...,...
8392,2018-09-28,Czech Republic,1
8396,2018-09-28,Czech Republic,1
8413,2018-09-28,Czech Republic,1
8426,2018-09-28,Czech Republic,1


In [25]:
df_test_prepro

,id,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,duration,departure_time_formated,arrival_time_formated,...,arr_long,distance_km,expected_duration,delay_relative_to_expected,duration_ratio,target,Country,City,is_holiday,holiday_length
0,test_id_0,DJE,TUN,2016-05-04 06:40:00,2016-05-04 07:30:00,ATA,TU 32AIMF,3000.0,2016-05-04 06:40:00,2016-05-04 07:30:00,...,10.22720,33463,3600.0,-600.0,0.833333,NaN,Tunisia,Djerba,0,0
1,test_id_1,TUN,BKO,2016-05-05 15:20:00,2016-05-05 20:05:00,ATA,TU 320IMW,17100.0,2016-05-05 15:20:00,2016-05-05 20:05:00,...,-7.94994,325505,17100.0,0.0,1.000000,NaN,Tunisia,Tunis,0,0
2,test_id_2,FRA,TUN,2016-05-06 10:00:00,2016-05-06 12:25:00,ATA,TU 32AIMC,8700.0,2016-05-06 10:00:00,2016-05-06 12:25:00,...,10.22720,147123,8700.0,0.0,1.000000,NaN,Germany,Frankfurt,0,0
3,test_id_3,BEY,TUN,2016-05-11 09:40:00,2016-05-11 13:10:00,ATA,TU 31BIMO,12600.0,2016-05-11 09:40:00,2016-05-11 13:10:00,...,10.22720,230919,12600.0,0.0,1.000000,NaN,Lebanon,Beirut,0,0
4,test_id_4,ORY,MIR,2016-05-11 09:50:00,2016-05-11 12:35:00,ATA,TU 736IOQ,9900.0,2016-05-11 09:50:00,2016-05-11 12:35:00,...,10.75470,159661,9300.0,600.0,1.064516,NaN,France,Paris,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,test_id_9328,TUN,NCE,2018-09-12 14:15:00,2018-09-12 15:45:00,ATA,TU 320IMV,5400.0,2018-09-12 14:15:00,2018-09-12 15:45:00,...,7.21587,79877,5400.0,0.0,1.000000,NaN,Tunisia,Tunis,0,0
9329,test_id_9329,TUN,TUN,2018-09-27 22:00:00,2018-09-28 01:00:00,SCH,TU 32AIMG,10800.0,2018-09-27 22:00:00,2018-09-28 01:00:00,...,10.22720,0,10800.0,0.0,1.000000,NaN,Tunisia,Tunis,0,0
9330,test_id_9330,SJJ,TUN,2018-09-03 09:20:00,2018-09-03 11:10:00,SCH,TU CR9ISA,6600.0,2018-09-03 09:20:00,2018-09-03 11:10:00,...,10.22720,103486,6600.0,0.0,1.000000,NaN,Bosnia and Herzegovina,Sarajevo,0,0
9331,test_id_9331,TUN,DJE,2018-09-15 14:30:00,2018-09-15 15:30:00,SCH,UG AT7LBD,3600.0,2018-09-15 14:30:00,2018-09-15 15:30:00,...,10.77550,33463,3600.0,0.0,1.000000,NaN,Tunisia,Tunis,0,0


In [26]:
df_test_prepro.to_csv('data/test_data_with_everything_final.csv', index=False)